## Domácí úkol - Batoh

Za domácí úkol budete mít vyřešit pomocí evolučního algoritmu problém batohu. Ten spočívá v tom, že máme batoh kapacity K a N předmětů, každý s cenou c<sub>i</sub> a objemem v<sub>i</sub> a chceme vybrat takové věci, abychom maximalizovali zisk a zároveň abychom nepřekročili kapacitu batohu. 

Vstupní data máte ve složce *domaci_ukol_data*. Obsahuje čtyři soubory s daty a dva s výsledky. Na první řádce souboru s daty je vždy počet předmětů a kapacita batohu oddělené mezerou, každý další následující řádek obsahuje cenu a objem předmětu taktéž oddělené mezerou. První dva soubory slouží pro snažší odladění evolučního algoritmu a obsahují i k sobě extra soubory s optimálním řešením. Na dalších dvou máte za úkol algoritmus pustit a výsledky na nich naměřené mi poslat. 

Napište tedy nějaký svůj evoluční algoritmus, který bude řešit problém batohu a pusťte ho na vstupních datech. Svůj kód, popis evolučního algoritmu (zvolené evoluční operátory, kódování jedince, atd.) a rozbor výsledků, včetně nejlepšího dosaženého skóre i s jejich odůvodněním mi pošlete emailem do stanoveného deadline.  Pro sepsání popisu vašeho evolučního algoritmu, parametrů evoluce, zvolené reprezentace jedince a rozboru výsledků použijte [tento template](https://github.com/kackamac/Prirodou-inspirovane-algoritmy/blob/master/04_spojita_reprezentace/DU1_evolucni_algoritmy.pdf).

##### Importy

In [1]:
import array
import random
import numpy as np
import math

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

import my_algorithms

Inicializácia random itemov a veľkosť batohu

In [2]:
# random.seed(64)
random.seed()
items_count = 20

weight_bounds = (10,200)
price_bounds = (10,200)

weights = random.sample(range(weight_bounds[0],weight_bounds[1]), items_count)
prices = random.sample(range(price_bounds[0],price_bounds[1]), items_count)

max_weight = 1000

Načítanie batohu zo súboru

In [3]:
#import weights and prices from txt file

def load_input_data(file_path):

    weights = []
    prices = []

    with open(file_path) as file:
        line_split = file.readline().split(" ")
        items_count = int(line_split[0])
        max_weight = int(line_split[1])
        for i in range(items_count):
            line_split = file.readline().split(" ")
            prices.append(int(line_split[0]))
            weights.append(int(line_split[1]))

    return weights, prices, max_weight

### Sedond try - permutation individual

max fitness - berieme itemy v poradí permutácie, kým nenaplníme batoh. Ostatné neberieme

In [4]:
def fitness_batoh_perm(individual):
    total_price = 0
    total_weight = 0
    for i in individual:
        total_weight += weights[i]
        if total_weight > max_weight:
            return total_price,
        total_price += prices[i]
    return total_price,

In [5]:
def fitness_batoh_perm_2(individual):
    total_price = 0
    total_weight = 0
    for i in individual:
        total_weight += weights[i]
        if total_weight <= max_weight:
            total_price += prices[i]
        else:
            total_weight -= weights[i]
    return total_price,

In [6]:
def mut_transposition(individual, indpb, indv_size):
    a = random.randint(0, indv_size - 1)
    b = random.randint(0, indv_size - 1)

    if random.random() < indpb:
        individual[a], individual[b] = individual[b], individual[a]

    return individual,

In [7]:
weights, prices, max_weight = load_input_data(file_path = "./domaci_ukol_data/input_data_1000.txt")
indv_size = len(weights)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("indices", random.sample, range(indv_size), indv_size)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness_batoh_perm)

toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", mut_transposition, indpb = 1, indv_size=indv_size)
toolbox.register("select", tools.selRoulette)

In [8]:
POP_SIZE = 1000
GEN_COUNT = 2000


pop = toolbox.population(n=POP_SIZE)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = my_algorithms.eaSimple(pop, toolbox, mu=POP_SIZE, cxpb=0.6, mutpb=0.3, ngen=GEN_COUNT,
                               stats=stats, halloffame=hof, verbose=True)

print(hof)
print(fitness_batoh_perm(hof[0]))

gen	nevals	avg    	std    	min	max 
0  	1000  	4590.73	1245.46	822	8688
1  	714   	5557.54	1413.22	2108	8831
2  	712   	6268.82	1534.2 	1522	10181
3  	729   	6917.41	1579.83	2630	10181
4  	704   	7505.97	1589.81	2697	10452
5  	723   	7919.63	1745   	2896	10643
6  	723   	8449.45	1776.71	3308	11510
7  	742   	8820.74	1822.97	3582	12426
8  	730   	9218.48	1912.85	2094	12426
9  	734   	9677.36	1915.74	3446	14021
10 	718   	10188.9	1867.88	3013	14021
11 	683   	10533.1	1983.98	3929	14021
12 	735   	10847.9	2066.03	3680	14021
13 	722   	11224.7	2071.34	3530	14021
14 	721   	11602.7	2188.44	3877	14110
15 	706   	11947.2	2349.01	3912	14396
16 	721   	12340.1	2333.65	2936	14396
17 	733   	12900.5	1921.76	4466	14739
18 	724   	13235.6	1698.26	4815	14746
19 	720   	13551.8	1480   	3140	15941
20 	729   	13616.3	1334.96	5481	15941
21 	735   	13763.9	1318.15	8386	15941
22 	716   	13923.2	1356.6 	8386	16101
23 	727   	14207.5	1291.79	8005	16266
24 	714   	14379.7	1465.43	9532	16377
25 	712   	14492.

In [9]:
print(hof)
print(fitness_batoh_perm(hof[0]))

[[419, 60, 494, 281, 585, 523, 480, 10, 739, 821, 822, 273, 886, 215, 703, 824, 469, 37, 989, 836, 13, 736, 216, 830, 446, 30, 732, 362, 121, 38, 708, 476, 379, 751, 845, 657, 32, 610, 914, 770, 855, 146, 967, 987, 493, 945, 347, 603, 23, 599, 249, 48, 373, 25, 737, 53, 426, 382, 937, 134, 743, 473, 573, 612, 421, 786, 992, 254, 775, 334, 6, 463, 269, 986, 245, 669, 903, 218, 462, 594, 307, 876, 704, 561, 214, 923, 791, 391, 435, 782, 74, 294, 157, 749, 67, 486, 919, 290, 746, 910, 108, 816, 692, 93, 355, 950, 965, 747, 182, 705, 864, 141, 819, 779, 130, 530, 781, 336, 319, 752, 321, 626, 902, 856, 981, 767, 104, 983, 410, 633, 638, 607, 514, 284, 938, 209, 582, 978, 96, 590, 41, 837, 899, 76, 729, 250, 208, 478, 467, 892, 867, 370, 289, 318, 616, 900, 206, 470, 809, 839, 62, 551, 940, 366, 663, 155, 636, 201, 921, 453, 305, 360, 398, 50, 712, 865, 128, 961, 465, 872, 456, 268, 563, 776, 39, 524, 861, 440, 219, 98, 642, 798, 578, 139, 879, 310, 728, 149, 941, 438, 186, 721, 780, 272, 2

#### first try - binárny jedinci

max fitness (cena ju zvyšuje a váha prekročená cez max nosnosť batohu prináša postihy)

In [ ]:
def fitness_batoh_bin(individual):
    total_price = np.sum(np.array(individual) @ np.array(prices))
    total_weight = np.sum(np.array(individual) @ np.array(weights))

    if total_weight <= max_weight:
        return total_price,
    else:
        return 1,

Vytvorenie prostredia (knižnica Deap)

In [ ]:
weights, prices = load_input_data(file_path = "./domaci_ukol_data/input_data_100.txt")
indv_size = len(weights)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", array.array, typecode='b', fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, indv_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_batoh_bin)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.2)
toolbox.register("select", tools.selRoulette)

Final part

In [ ]:
pop = toolbox.population(n=500)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=200, 
                               stats=stats, halloffame=hof, verbose=True)